# Model Interpretability Tool Demo

This demo shows how LIME and SHAP can be used to explain a model (kernel SVM). 

For the demo we use the "What's Cooking?" dataset from Kaggle (more info [here](https://www.kaggle.com/c/whats-cooking/data)). The dataset is a list of ingredients for a recipe identified by the type of cuisine. The goal is to train a model to predict the cuisine from the ingredients list.


### Resources
The following resources were used for the installation and use of the tools
* [LIME tutorial](https://github.com/marcotcr/lime/blob/master/doc/notebooks/Lime%20-%20basic%20usage%2C%20two%20class%20case.ipynb) by Ribeiro, Singh, and Guestrin
* [SHAP tutorial](https://github.com/slundberg/shap) by Lundberg and Lee

To get started we will import our model explanation tools (please install using the links above) and scikit-learn

In [1]:
import pickle
import pandas as pd
import shap
import re
from random import randint
from lime import lime_text
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

Now we load and process our dataset, which consists of a list of ingredients identified by a type of cuisine. The dataset comes in JSON form with ingredients as a list; this form does not work well with our model explaners (they expect raw text), so we will do a simple conversion from list to text before moving on.

Note we do not use the test data provided by Kaggle because it does not have labels. Instead we will just split the provided training data.

In [2]:
# Load data
data_df = pd.read_json("data/train.json").set_index("id")

# Create string from ingredients list
def clean_ingredients(ingredient_list):
    s = ""
    for i in ingredient_list:
        i = re.sub("-|\s", "_", i.strip())
        s += f"{i} "
    return s[:-1]
data_df.ingredients = data_df.ingredients.map(lambda x: clean_ingredients(x))

# Encode labels
le = LabelEncoder()
data_df["target"] = le.fit_transform(data_df.cuisine)

# Split into train and test
train_df = data_df.sample(frac=0.7)
test_df = data_df[~data_df.index.isin(train_df.index)]

Here is an example data point. We have a cuisine type and the (string) of ingredients in that recipe.

In [3]:
train_df.iloc[0]

cuisine                                        southern_us
ingredients    water salt butter baking_powder heavy_cream
target                                                  16
Name: 42797, dtype: object

As features for our model we will use TFIDF for no reason other than it's simple.

In [4]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_df.ingredients)
X_test = vectorizer.transform(test_df.ingredients)

In [5]:
# Check our vectorizer
print(f"There are {len(vectorizer.get_feature_names())} unique tokens")
# vectorizer.get_feature_names()

There are 6186 unique tokens


Time to train our model!

## Model Creation

Let's initialize our kernel SVM. Please note the large cache size and change it if you do not have that much memory! Alternatively, do not re-train the model and simply load the one in this repo (it takes a bit to train).

In [6]:
# Train model
clf = SVC(kernel="rbf", gamma="scale", probability=True)
clf.fit(X_train, train_df.cuisine)

# Dump
pickle.dump(clf, open("model_svm.pickle", "wb"))

In [7]:
# Load model
clf = pickle.load(open("model_svm.pickle", "rb"))

Model trained! Now we can check our model train and test accuracy

In [8]:
train_score = precision_recall_fscore_support(train_df.target, 
                                                clf.predict(X_train), 
                                                average="weighted")
test_score = precision_recall_fscore_support(test_df.target, 
                                                clf.predict(X_test), 
                                                average="weighted")
print(f"Train F1: {train_score[2]}")
print(f"Test F1: {test_score[2]}")

ValueError: Mix of label input types (string and number)

## Model Explanation with Feature Importance

...gotcha! In this example we are using a kernel (non-linear) SVM. 

## Model Explanation with LIME

We have our training and test F1 scores, now let's explore our model with LIME. The goal here is to understand which features our model is picking up on, and seeing how we can improve our feature engineering.

In [13]:
# Create sklearn pipeline as in LIME tutorial
pipe = make_pipeline(vectorizer, clf)

# Initialize LIME
class_names = train_df.cuisine.unique().tolist()
class_names.sort()
explainer_lime = lime_text.LimeTextExplainer(class_names=class_names)

In [14]:
# Select a sample
idx = 0
test_sample = test_df.iloc[idx]
true_label, pred_label = test_sample.target, clf.predict(X_test[idx])[0]
true_cuisine, pred_cuisine = le.inverse_transform([true_label, pred_label])
print(f"The correct class is {true_cuisine} and the model predicted {pred_cuisine}")

ValueError: y contains previously unseen labels: ['british']

In [ ]:
num_features = 20
# Show explanation for the correct class and another class
if true_label == pred_label:
    labels = [true_label, randint(0,len(le.classes_))]
else:
    labels = [true_label, pred_label]
exp = explainer_lime.explain_instance(test_sample.ingredients, 
                                 pipe.predict_proba, 
                                 num_features=num_features,
                                    labels=labels)
fig = exp.as_pyplot_figure(label=labels[0])
exp.show_in_notebook(text=True)

## Model Explanation with SHAP

In [ ]:
explainer_shap = shap.KernelExplainer(clf.predict_proba, X_train[:100], link="logit")

In [ ]:
shap_values = explainer_shap.shap_values(X_test[idx])

In [ ]:
# plot the SHAP values for the last used test sample in LIME, for comparison
shap.force_plot(explainer_shap.expected_value[idx], shap_values[idx][0,:], X_test.iloc[idx,:], link="logit")